# 特征

2021年9月16日，漱玉平民出现低位高换手，换手率18.4%，上涨4.4%，上影线5.6%。日线量比达4.8，30分钟量比更是高达29.5倍。其后不久，连续出现20cm涨停。

又如上声电子， 2021-11-03前后出现上述特征，随后一周上涨50%以上。

需要直接使用jqdatasdk

买入点特征：
1. 新股或者次新股（<1年）
2. 换手率15%以上
3. 连续两日以上涨幅5%以上
4. 低位: 120日altitude < 0.3
5. RSI？

In [62]:
from alpha.notebook import *
from alpha.core.rsi_stats import rsiday, rsi30
import datetime
import os

await init_notebook(adaptor='both')

# 高换手版（适用于新股？）

In [63]:
def features(code, name, bars, results, frame_type):
    altitude_threshold =  0.6
    lb_threshold =  3
    roc_threshold = 0.05
    
    high = bars["high"]
    close = bars["close"]
    low = bars["low"]
    open_ = bars["open"]

    volume = bars["volume"]

    c2, c1, c0 = close[-3:]
    v2, v1, v0 = volume[-3:]
    
    lv = np.min(volume[-6:-1])
    
    hh = np.max(high)
    ll = np.min(low)
    
    altitude = 1 - (hh - close[-1])/(hh - ll)
    
    roc2, roc1, roc0 = close[-3:] / close[-4:-1] - 1
    
    to2 = jq_get_turnover_realtime(code, v2, c2)
    to1 = jq_get_turnover_realtime(code, v1, c1)
    to0 = jq_get_turnover_realtime(code, v0, c0)
    
    if len(days) < 60:
        return
    
    results.append([name, code, altitude, roc2, roc1, roc0, to2, to1, to0, days])

results = await scan(features, 60, nstocks=-1, silent=True)

progress: 500/4002, elapsed: 2, ETA: 14
progress: 1000/4002, elapsed: 5, ETA: 15
progress: 1500/4002, elapsed: 7, ETA: 11
progress: 2000/4002, elapsed: 9, ETA: 9
progress: 2500/4002, elapsed: 12, ETA: 7
progress: 3000/4002, elapsed: 14, ETA: 4
progress: 3500/4002, elapsed: 16, ETA: 2
progress: 4000/4002, elapsed: 18, ETA: 0


In [19]:
results

[]

# 高量比版

参考标的：翰宇药业， 2021年11月9日前后

1. 量比极高
2. 平台突破

In [64]:
def features(code, name, bars, results, frame_type):
    close = bars["close"]
    volume = bars["volume"]
    high = bars["high"]
    low = bars["low"]
    
    c1, c0 = close[-2:]
    v0 = volume[-1]
    
    if c0/c1 - 1 < 0:
        return
    
#     mas = []
#     for win in [5, 10, 20, 30, 60]:
#         mas.append(moving_average(close, win)[-20:])
        
#     mas_max = np.max(mas, axis=0)
#     mas_min = np.min(mas, axis=0)
#     convergency = 1 - mas_min/mas_max
    
#     # count of convergency
#     coc = np.count_nonzero(convergency < 0.1)/20
    
    hh = np.max(high)
    ll = np.min(low)
    
    # altitude
    alt = 1 - (hh - c0)/(hh - ll)
    
    # 量比
    lb = v0 / np.min(volume[-10:])
    
    results.append((name, code, alt, lb))

In [65]:
results = await scan(features, 120, nstocks=-1, tm="2021-11-01", silent=True)
df = pd.DataFrame(results, columns=["name", "code", "alt", "lb"])

progress: 500/4002, elapsed: 1, ETA: 7


2021-11-13 02:00:41,799 E 34623 asyncio:default_exception_handler:1707 | Task was destroyed but it is pending!
task: <Task pending name='Task-5' coro=<RedisConnection._read_data() running at /home/aaron/miniconda3/envs/alpha/lib/python3.8/site-packages/aioredis/connection.py:186> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0xffff590f0af0>()]> cb=[RedisConnection.__init__.<locals>.<lambda>() at /home/aaron/miniconda3/envs/alpha/lib/python3.8/site-packages/aioredis/connection.py:168]>
2021-11-13 02:00:41,799 E 34623 asyncio:default_exception_handler:1707 | Task was destroyed but it is pending!
task: <Task pending name='Task-6' coro=<RedisConnection._read_data() running at /home/aaron/miniconda3/envs/alpha/lib/python3.8/site-packages/aioredis/connection.py:186> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0xffff0f3b97c0>()]> cb=[RedisConnection.__init__.<locals>.<lambda>() at /home/aaron/miniconda3/envs/alpha/lib/python3.8/site-packages/aioredis/connecti

progress: 1000/4002, elapsed: 4, ETA: 12
progress: 1500/4002, elapsed: 5, ETA: 8
progress: 2000/4002, elapsed: 7, ETA: 7
progress: 2500/4002, elapsed: 9, ETA: 5
progress: 3000/4002, elapsed: 11, ETA: 3
progress: 3500/4002, elapsed: 13, ETA: 1
progress: 4000/4002, elapsed: 14, ETA: 0


In [66]:
df.nlargest(10, "lb")

,name,code,alt,lb
2547,文山电力,600995.XSHG,1.000000,138.672208
361,青岛食品,001219.XSHE,0.976982,103.142078
358,华瓷股份,001216.XSHE,0.407839,61.124959
2488,北京城乡,600861.XSHG,0.382923,44.051297
2000,四川路桥,600039.XSHG,0.953061,35.046997
978,崇达技术,002815.XSHE,1.000000,31.374755
3130,同力日升,605286.XSHG,1.000000,26.664452
1892,英力股份,300956.XSHE,0.478599,26.124221
2288,天下秀,600556.XSHG,1.000000,25.624802
1410,新文化,300336.XSHE,0.854167,24.416701


# V1 低位放量

In [ ]:
async def backtest_v1(start, end):
    start = arrow.get(start).date()
    end = arrow.get(end).date()
    
    signals = []
    for code in Securities().choose(["stock"]):
        sec = Security(code)
        try:
            bars = await sec.load_bars(start, end, FrameType.DAY)

            for i in range(120, len(bars) - 15):
                xbars = bars[i-120:i]

                signal = fire_long(code, xbars, pcr=0.2)
                if signal is None:
                    continue
                else:
                    frame, price, altitude, lb, to, pcr_0 = signal
                    row = [sec.display_name, code, frame, price, altitude, lb, to, pcr_0]
                    for j in [1, 3, 5, 10, 15]:
                        ybars = bars[i:i+j]
                        yclose = np.max(ybars["close"])
                        row.append(yclose/price-1)

                    signals.append(row)
        except Exception:
            pass
        
    columns = ["name", "code", "frame", "price", "altitude", "lb", "to", "pcr_0", 
               "pcr_1", "pcr_3", "pcr_5", "pcr_10", "pcr_15"]
    return pd.DataFrame(signals, columns=columns)
        
df = await backtest_v1("2021-01-04", "2021-09-30")
df

In [ ]:
df.describe()

# V2 加上次新股属性

回测表明，次新股低位放量后，10交易日平均涨幅为12%；15日平均涨幅为15%。

In [ ]:
async def backtest_v2(start, end, **params):
    end = arrow.get(end).date()
    start = arrow.get(start).date()
    
    signals = []
    for code in Securities().choose(["stock"]):
        sec = Security(code)
        
        if tf.day_shift(sec.ipo_date, 60) < start:
            continue
            
        try:
            bars = await sec.load_bars(sec.ipo_date, end, FrameType.DAY)

            for i in range(60, min(250, len(bars) - 15)): #限制在一年内
                xbars = bars[i-60:i]

                signal = fire_long(code, xbars, **params)
                if signal is None:
                    continue
                else:
                    frame, price, altitude, lb, to, pcr_0 = signal
                    row = [sec.display_name, code, frame, price, altitude, lb, to, pcr_0]
                    for j in [1, 3, 5, 10, 15]:
                        ybars = bars[i:i+j]
                        yclose = np.max(ybars["close"])
                        row.append(yclose/price-1)

                    signals.append(row)
        except Exception:
            pass
        
    columns = ["name", "code", "frame", "price", "altitude", "lb", "to", "pcr_0", 
               "pcr_1", "pcr_3", "pcr_5", "pcr_10", "pcr_15"]
    return pd.DataFrame(signals, columns=columns)

In [ ]:
df = await backtest_v2("2015-01-04", "2021-09-30", altitude=0.6, turnover=0.07, lb=3, pcr=0.05)
df.describe()

In [ ]:
report("低位放量v2次新股版回测结果", {"altitude":0.6, "turnover":0.07, "lb":3, "pcr":0.05}, result_df=df)

# v3
放量后，缩量下跌至低于放量涨当日开盘价买入，计算持有1，3，5，10，15日，以收盘价卖出的利润。结果表明并无改进。

In [ ]:
async def backtest_v3(start, end):
    start = arrow.get(start).date()
    end = arrow.get(end).date()
    
    signals = []
    
    # 寻找发出放量信号的个股
    for code in Securities().choose(["stock"]):
        sec = Security(code)
        try:
            bars = await sec.load_bars(start, end, FrameType.DAY)

            for i in range(120, len(bars) - 30):
                xbars = bars[i-120:i]

                signal = fire_long(code, xbars, pcr=0.3, altitude=0.8, lb=2.5)
                if signal is None:
                    continue
                else:
                    frame, close, altitude, lb, to, pcr_0 = signal
                    row = [code, frame, altitude, lb, to]
                    signals.append(row)
        except Exception:
            pass
        
    # 如果股价回到放量日开盘价以下，则为买入点
    results = []
    for code, sig_start, altitude, lb, to in signals:
        sig_end = tf.day_shift(sig_start, 15)
        sec = Security(code)
        try:
            bars = await sec.load_bars(sig_start, sig_end, FrameType.DAY)
            open_price = bars["open"][0]
            
            sig_bars = bars[1:]
            pos = np.argwhere(sig_bars["low"] <= open_price).flatten()
            if len(pos) == 0:
                continue
                
            order_day = sig_bars[pos[0]]["frame"]
            ystart = tf.day_shift(order_day, 1)
            yend = tf.day_shift(order_day, 16)
            
            ybars = await sec.load_bars(ystart, yend, FrameType.DAY)
            
            row = [sec.display_name, code, sig_start, order_day, open_price, altitude, lb, to]
            for n in [1, 3, 5, 10, 15]:
                c = ybars[n]["close"] / open_price - 1
                row.append(c)
            
            results.append(row)
        except Exception as e:
            break
        
    columns = ["name", "code", "fire_date", "oder_date", "order_price", "altitude", "lb", "to",  
               "pcr_1", "pcr_3", "pcr_5", "pcr_10", "pcr_15"]
    return pd.DataFrame(results, columns=columns)
        
df = await backtest_v3("2021-01-04", "2021-09-30")
df

# V4
找出近10天日线放量最大超过4倍，或者换手率超过20%的标的，30分钟线出现rsi低于25%时发出提示。

In [13]:
def daybars_trigger(name, code, bars, results, frame_type):
    n = 10
    last_n = bars[-n:]
    volume = last_n["volume"]
    vr = max(volume)/min(volume)
#     if vr < 4:
#         return
    
    pos = np.argmax(volume)
    flag = last_n[pos]["close"] < last_n[pos]["open"] #阴线放量
    
    results.append([name, code, vr, n-pos, flag])
    
result = await scan(daybars_trigger, 15, tm="2021-11-05")

In [14]:
df = pd.DataFrame(result, columns=["code", "name", "vr", "dist", "flag"])
pd.options.display.max_rows=999
df.nlargest(10, "vr")

,code,name,vr,dist,flag
37,000055.XSHE,方大集团,11.171302,8,True
21,000030.XSHE,富奥股份,10.557848,3,False
97,000505.XSHE,京粮控股,9.476838,2,False
58,000151.XSHE,中成股份,8.553046,1,True
28,000038.XSHE,深大通,7.873495,8,True
74,000407.XSHE,胜利股份,6.198284,8,False
15,000023.XSHE,深天地A,5.955041,1,False
12,000019.XSHE,深粮控股,5.887014,3,False
20,000029.XSHE,深深房A,5.791306,10,False
43,000062.XSHE,深圳华强,5.686140,3,True


In [67]:
124.36*6.38

793.4168